In [1]:
import os
import re
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.setup_db import DatabaseConnection
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.memory.vector_search import MemoryRetriever
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment

In [2]:
# Load config
config_path = Path("../market_agents/memory/memory_config.yaml")
memory_config = load_config_from_yaml(str(config_path))

# Initialize database connection and embedding service
db_conn = DatabaseConnection(memory_config)
embedder = MemoryEmbedder(memory_config)

# Initialize knowledge base with "shakespeare" prefix
shakespeare_kb = MarketKnowledgeBase(
    config=memory_config,
    db_conn=db_conn,
    embedding_service=embedder,
    table_prefix="shakespeare_kb"
)

In [3]:
def read_txt_content(txt_path):
    with open(txt_path, 'r') as file:
        text_content = file.read()
    return text_content

def read_pdf_content(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text_content = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        cleaned_text = re.sub(r'FTLN \d+', '', page_text)
        cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
        text_content += cleaned_text
    return text_content

In [4]:

# Example usage for PDF
pdf_path = "/Users/interstellarninja/Documents/projects/MarketAgents/hamlet_PDF_FolgerShakespeare.pdf"
shakespear_bib = read_pdf_content(pdf_path)

# Example usage for TXT
txt_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/the_complete_works_of_william_shakespear.txt"
hamlet_text = read_txt_content(txt_path)

# Store in knowledge base with metadata
knowledge_id = shakespeare_kb.ingest_knowledge(
    shakespear_bib,
    metadata={
        #"source": "Hamlet - Folger Shakespeare Library",
        "source": "The Complete Works of William Shakespeare",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")

['Get even mor e from the Folger\nYou can get your own copy of this text to keep. Purchase a full copy\nto get the text, plus explanatory notes, illustrations, and more. Buy a copyFolger Shakespeare Library\nhttps://shakespeare.folger.edu/Front\nMatterFrom the Director of the Folger Shakespeare\nLibrary\nTextual Introduction\nSynopsis\nCharacters in the Play\nACT 1Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nACT 2Scene 1\nScene 2\nACT 3Scene 1\nScene 2\nScene 3\nScene 4\nACT 4Scene 1\nScene 2\nScene 3\nScene 4\nScene 5\nScene 6\nScene 7\nACT 5Scene 1\nScene 2ContentsMichael Witmore\nDirector, Folger Shakespeare LibraryIt is hard to imagine a world without Shakespeare.', 'Since their\ncomposition four hundred years ago, Shakespeare’s plays and poems\nhave traveled the globe, inviting those who see and read his works to\nmake them their own. Readers of the New Folger Editions are part of this ongoing process\nof “taking up Shakespeare,” finding our own thoughts and feelings\nin language 

In [5]:
# Initialize retriever
retriever = MemoryRetriever(memory_config, db_conn, embedder)

# Create knowledge base agent
kb_agent = KnowledgeBaseAgent(
    market_kb=shakespeare_kb,
    retriever=retriever
)

In [6]:

# Test some queries
def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = kb_agent.retrieve(question, "shakespeare_kb")
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    query_hamlet(query)


Question: What is Hamlet's famous 'To be or not to be' soliloquy about?
--------------------------------------------------------------------------------
["What is Hamlet's famous 'To be or not to be' soliloquy about?"]

Result 1 (Similarity: 0.824)
Text: SC. 2
HORATIO
Sound a flourish. HAMLET
KING
HAMLET
KING
HAMLET That they are not a pipe for Fortune’s finger
 To sound what stop she please. Give me that man
 That is not passion’s slave, and I will wear him
 In my heart’s core, ay, in my heart of heart,
 As I do thee.—Something too much of this.—
 There is a play tonight before the King. One scene of it comes near the circumstance
 Which I have told thee of my father’s death. I prithee, when thou seest that act afoot,
 Even with the very comment of thy soul
 Observe my uncle.

Context: ...owhere but in ’s own house. Farewell.
  O, help him, you sweet heavens!
  If thou dost marry, I’ll give thee this plague
 for thy dowry: be thou as chaste as ice, as pure as
 snow, thou shalt not es

In [7]:
import json


chat_env = ChatEnvironment(name="shakespeare_chat")

shakespeare_agent = MarketAgent.create(
    memory_config=memory_config,
    db_conn=db_conn,
    agent_id="shakespeare_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
)

# Add a test message to the chat
shakespeare_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
#shakespeare_agent.task = "What is the role of the ghost in the play?"
chat_env.mechanism.add_user_message(shakespeare_agent.task)

# Generate a response
perception = await shakespeare_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await shakespeare_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await shakespeare_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))



Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "literary_agent_01", "action": {"thoughts": [{"reasoning": "First, I need to recall the specific details of Ophelia\'s death in \'Hamlet\'. In the play, Ophelia\'s death is reported by Queen Gertrude in Act 4, Scene 7. Gertrude describes how Ophelia fell into a stream and drowned, suggesting it was an accident, but the ambiguity leaves room for interpretation as a possible suicide."}, {"reasoning": "Next, I should consider the thematic implications of Ophelia\'s death. Her demise is closely linked to themes of madness and tragedy. Ophelia\'s mental state deteriorates following her father\'s murder by Hamlet and Hamlet\'s rejection of her. This reflects the immense pressure she faces from her family and the court."}, {"reasoning": "Now, I should address how Ophelia\'s death is interpreted in various adaptations and analyses. Some interpretations view her death as a symbol of the oppression of wome

In [8]:
print(json.dumps(action['content'], indent=4))

{
    "agent_id": "literary_analysis_agent_001",
    "action": {
        "thoughts": [
            {
                "reasoning": "The user is asking about Hamlet's famous 'To be or not to be' soliloquy, which is a key moment in Shakespeare's play. This soliloquy is known for its exploration of existential themes, particularly the nature of existence, life, and death. The user might be looking for a deeper understanding of these themes as they relate to the play and possibly to broader philosophical ideas."
            },
            {
                "reasoning": "In order to provide a comprehensive response, I should explain the soliloquy's focus on Hamlet's internal struggle with the idea of existence and the fear of the unknown in death. Additionally, I should connect these ideas to the play's broader themes such as madness and indecision, which are central to Hamlet's character."
            },
            {
                "reasoning": "To engage the user further, I can pose thou

In [9]:
print(action['content']['action']['final_answer'])

Hamlet's 'To be or not to be' soliloquy is a profound exploration of existence and the nature of life and death. In this soliloquy, Hamlet contemplates the pain and unfairness of life versus the fear and uncertainty of death, which he describes as 'the undiscovered country'. This reflects his internal struggle with indecision and existential angst, themes that are central to the play. The soliloquy also touches on broader issues such as madness and the human condition. To deepen your understanding, consider how Hamlet's questioning of existence compares to modern existential thought, or how it might relate to other literary works that explore similar themes. What do you think Shakespeare is saying about the human experience through Hamlet's musings?
